# Modelo de Prediccion de ciudades.

Importamos las librerias necesarias para el modelo.
Obtenemos los datos de la base de datos de BigQuery y los cargamos en Pandas.

In [2]:
import os
import pandas as pd
from google.cloud import bigquery
import warnings
warnings.filterwarnings('ignore')
def run_query(query) -> pd.DataFrame:
    # Runs a BigQuery query and returns the results as a pandas DataFrame.
    query_job = client.query(query)
    return query_job.to_dataframe()

# Cargamos el archivo de credenciales de nuestro google cloud en una variable de entorno del PC.
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "braided-grammar-430922-b4-64d668a5dc36.json"

# Cargamos el proyecto y el cliente.
project_id = "braided-grammar-430922"
client = bigquery.Client()
id_exists = os.path.exists('data/dataML.json')

# Conseguimos los datos de las empresas.
query = "select * from datanexus.google_yelp_business"
data = run_query(query)
data.head()

,name,address,gmap_id,latitude,longitude,category,rating,review_count,state,postal_code,city
0,Fire Monkey Food Truck,,MxSSESMG5NJCDHy_zsmNCw,27.751828,-82.626734,"Food, Street Vendors, Caribbean, Food Trucks, ...",4.0,6,FL,,Saint Petersburg
1,"Parantha Alley, LLC",LLC,0x89c259852b3e5c83:0xf7e8364197e0bde7,40.726665,-73.992070,"['indian restaurant', 'restaurant']",4.3,4,NY,NY,New York
2,Nueces Bay Causeway,Corpus Christi,0x86685c14db61522b:0xd81b6521f24d7a61,27.844659,-97.369239,"['bridge', 'restaurant']",4.4,18,TX,TX,Corpus Christi
3,People's Pizza,1500 Rt 38,GIRFpGtBBrUuXt8k9yUf6Q,39.937769,-75.035172,"Sandwiches, Pizza, Restaurants",4.0,105,PA,08002,Cherry Hill
4,Wawa,500 New Jersey Highway 38,uWJ8yes4D6f7e0bUQdBeJg,39.936435,-75.048503,"Food, Coffee & Tea, Sandwiches, Restaurants, A...",2.5,14,PA,08002,Cherry Hill


Eliminamos las columnas innecesarias

In [3]:
data.drop(columns=['name', 'address', 'gmap_id', 'latitude', 'longitude', 'postal_code'], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41098 entries, 0 to 41097
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   category      41098 non-null  object 
 1   rating        41098 non-null  float64
 2   review_count  41098 non-null  Int64  
 3   state         41098 non-null  object 
 4   city          41098 non-null  object 
dtypes: Int64(1), float64(1), object(3)
memory usage: 1.6+ MB


Hacemos las normalizaciones de la columna categoria.

In [4]:
data['category'] = data['category'].apply(lambda x: [cat.strip().title() for cat in eval(x) if isinstance(x, str)] 
                                          if isinstance(x, str) and x.startswith('[') 
                                          else [cat.strip().title() for cat in x.split(',')])
data.head()

,category,rating,review_count,state,city
0,"[Food, Street Vendors, Caribbean, Food Trucks,...",4.0,6,FL,Saint Petersburg
1,"[Indian Restaurant, Restaurant]",4.3,4,NY,New York
2,"[Bridge, Restaurant]",4.4,18,TX,Corpus Christi
3,"[Sandwiches, Pizza, Restaurants]",4.0,105,PA,Cherry Hill
4,"[Food, Coffee & Tea, Sandwiches, Restaurants, ...",2.5,14,PA,Cherry Hill


Creamos una lista de las categorias que no son de comida.

In [5]:
categorias_no_comida = [
    "Acupuncture", "Active Life", "Aerial Fitness", "Airport Lounges", 
    "Allergy Testing", "Amusement Parks", "Animal Physical Therapy", 
    "Animal Shelters", "Apartments", "Aquarium Services", "Arcades", 
    "Art Classes", "Art Galleries", "Astrologers", "Auto Detailing", 
    "Auto Parts & Supplies", "Automotive", "Baby Gear & Furniture", 
    "Balloon Services", "Banks & Credit Unions", "Barbers", "Bartenders", 
    "Beauty & Spas", "Bed & Breakfast", "Bingo Halls", 
    "Blood & Plasma Donation Centers", "Boat Dealers", "Bookstores", 
    "Bowling", "Brazilian Jiu-jitsu", "Breweries", "Bridal", 
    "Campgrounds", "Car Dealers", "Car Wash", "Carpeting", "Casinos", 
    "Child Care & Day Care", "Churches", "Cinemas", "Civil Engineering", 
    "Climbing", "Coffee & Tea Supplies", "Comic Books", 
    "Community Service/Non-Profit", "Cosmetology Schools", 
    "Counseling & Mental Health", "Country Clubs", "Dance Schools", 
    "Day Camps", "Dentists", "Department Stores", "Dermatologists", 
    "Digital Marketing", "Disc Golf", "Dive Bars", 
    "Divorce & Family Law", "Drugstores", "Dry Cleaning & Laundry", 
    "Education", "Electricians", "Emergency Pet Hospital", 
    "Employment Agencies", "Entertainment Law", "Escape Games", 
    "Eyewear & Opticians", "Farmers Market", "Fashion", 
    "Financial Services", "Firearm Training", "Fitness & Instruction", 
    "Florists", "Football", "Foundation Repair", "Freelance Photographers", 
    "Furniture Stores", "Garage Door Services", "Gardening Centers", 
    "Gas Stations", "General Contractors", "Golf", "Gymnastics", 
    "Hair Removal", "Hair Salons", "Hardware Stores", "Health Markets", 
    "Hobby Shops", "Home & Garden", "Home Cleaning", "Home Health Care", 
    "Horse Boarding", "Horse Racing", "Hospitals", "Hotels & Travel", 
    "Insurance", "Interior Design", "Internet Service Providers", 
    "It Services & Computer Repair", "Kids Activities", "Libraries", 
    "Marketing", "Martial Arts", "Massage Therapy", "Medical Spas", 
    "Men's Clothing", "Mobile Phone Repair", "Museums", 
    "Music Production Services", "Musical Instruments & Teachers", 
    "Neurologist", "Nightlife", "Noodles", "Office Cleaning", 
    "Oil Change Stations", "Optometrists", "Oral Surgeons", 
    "Organic Stores", "Orthopedists", "Outdoor Furniture Stores", 
    "Paint & Sip", "Painting", "Parks", "Party Bus Rentals", 
    "Party Supplies", "Patisserie/Cake Shop", "Pawn Shops", 
    "Pediatric Dentists", "Pet Boarding", "Pet Groomers", 
    "Pet Services", "Pet Sitting", "Pet Stores", 
    "Photography Stores & Services", "Pilates", "Plumbers", 
    "Podiatrists", "Professional Services", "Property Management", 
    "Public Services & Government", "Real Estate", 
    "Recording & Rehearsal Studios", "Rehearsal Spaces", 
    "Reptile Shops", "Resorts", "Restaurants", "RV Repair", 
    "Sandwiches", "Saunas", "Security Systems", "Self Storage", 
    "Sewing & Alterations", "Shipping Centers", "Shoe Repair", 
    "Shopping", "Skate Shops", "Skydiving", "Social Clubs", 
    "Solar Installation", "Specialty Schools", "Sporting Goods", 
    "Sports Clubs", "Sports Wear", "Stadiums & Arenas", "Steakhouses", 
    "Structural Engineers", "Surfing", "Swimming Lessons/Schools", 
    "Tattoo", "Tax Services", "Tea Rooms", "Team Building Activities", 
    "Tennis", "Towing", "Transmission Repair", "Travel Agents", 
    "Tree Services", "Universities", "Used, Vintage & Consignment", 
    "Veterinarians", "Video/Film Production", "Vinyl Records", 
    "Virtual Reality Centers", "Weight Loss Centers", 
    "Window Washing", "Wine Tasting Classes", "Yoga"
]


Filtramos las categorias que no son de comida de nuestro dataset.

In [6]:
data['category'] = data['category'].apply(lambda x: [cat for cat in x if cat not in categorias_no_comida])
data.head()

,category,rating,review_count,state,city
0,"[Food, Street Vendors, Caribbean, Food Trucks]",4.0,6,FL,Saint Petersburg
1,"[Indian Restaurant, Restaurant]",4.3,4,NY,New York
2,"[Bridge, Restaurant]",4.4,18,TX,Corpus Christi
3,[Pizza],4.0,105,PA,Cherry Hill
4,"[Food, Coffee & Tea, Convenience Stores]",2.5,14,PA,Cherry Hill


Exportamos los datos transformados.

In [8]:
# Guardamos los datos verificando que no exista el archivos
data.to_json('data/dataML.json', orient='records', lines=True)
print("Datos exportados existosamente")

Datos exportados existosamente


Importamos los datos ya transformados si tenemos el archivo.

In [9]:
import pandas as pd
data = pd.read_json('data/dataML.json', lines=True)
data.head()

,category,rating,review_count,state,city
0,"[Food, Street Vendors, Caribbean, Food Trucks]",4.0,6,FL,Saint Petersburg
1,"[Indian Restaurant, Restaurant]",4.3,4,NY,New York
2,"[Bridge, Restaurant]",4.4,18,TX,Corpus Christi
3,[Pizza],4.0,105,PA,Cherry Hill
4,"[Food, Coffee & Tea, Convenience Stores]",2.5,14,PA,Cherry Hill


Exploramos las categorías

In [10]:
data['category'].explode().unique().tolist()

['Food',
 'Street Vendors',
 'Caribbean',
 'Food Trucks',
 'Indian Restaurant',
 'Restaurant',
 'Bridge',
 'Pizza',
 'Coffee & Tea',
 'Convenience Stores',
 'Thai',
 'Chinese',
 'Soup',
 'American (New)',
 'Fast Food',
 'Burgers',
 'Chicken Wings',
 'Korean',
 'Food Delivery Services',
 'Wedding Planning',
 'Event Planning & Services',
 'Caterers',
 'Party & Event Planning',
 'Salad',
 'Desserts',
 'Seafood',
 'American (Traditional)',
 'Bars',
 'Italian',
 'Wine Bars',
 'Cafes',
 'Vietnamese',
 'Breakfast & Brunch',
 'Mexican',
 'Tacos',
 'Tex-Mex',
 'Vegetarian',
 'New Mexican Cuisine',
 'Gluten-Free',
 'Ice Cream & Frozen Yogurt',
 'Japanese',
 'Buffets',
 'Sushi Bars',
 'Diners',
 'Grocery',
 'Beer',
 'Wine & Spirits',
 'Flowers & Gifts',
 'Shopping Centers',
 'Cocktail Bars',
 'Venues & Event Spaces',
 'Lounges',
 'Mediterranean',
 'Sports Bars',
 'Irish',
 'Pubs',
 'Candy Stores',
 'Indian',
 'Specialty Food',
 'Ethnic Food',
 'Asian Fusion',
 'Donuts',
 'Nurseries & Gardening',


Creamos todo un sistema para el preprocesamiento, entrenamiento y predicción de nuestro modelo.

In [11]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

def filter_data(state, category):    
    return data[(data['state']  == state) &(data["category"].apply(lambda x: category in x))]

# Calcular competencia (cantidad de restaurantes en la misma categoría y ciudad)
def calculate_competition(data):
    competition = data.groupby('city').size().reset_index(name='competition')
    return data.merge(competition, on='city')

# Crear un proxy de éxito (puede ser popularidad o una combinación)
def calculate_success(data):
    data['success'] = data['rating'] * data['review_count']
    return data

# Pipeline de preprocesamiento y creación de características
def preprocess_data(state, category):
    filtered_data = filter_data(state, category)
    filtered_data = calculate_competition(filtered_data)
    filtered_data = calculate_success(filtered_data)
    return filtered_data

# Entrenar el modelo
def train_model(state, category):
    processed_data = preprocess_data(state, category)
    X = processed_data[['competition', 'rating', 'review_count']]  # Variables predictoras
    y = processed_data['success']  # Variable objetivo
    
    # Dividir en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Entrenar el modelo de Random Forest
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Hacer predicciones
    y_pred = model.predict(X_test)
    
    # Evaluar el modelo
    print("MSE:", mean_squared_error(y_test, y_pred))
    print("R2:", r2_score(y_test, y_pred))
    
    return model, processed_data

# Hacer la recomendación
def recommend_city(state, category):
    model, processed_data = train_model(state, category)
    predictions = model.predict(processed_data[['competition', 'rating', 'review_count']])
    processed_data['predicted_success'] = predictions
    best_city = processed_data.loc[processed_data['predicted_success'].idxmax()]['city']
    return best_city


state = 'FL'
category = 'Seafood'
best_city = recommend_city(state, category)
print(f"The best city to open a {category} restaurant in {state} is {best_city}.")

MSE: 118272.78601586541
R2: 0.8829303495192822
The best city to open a Seafood restaurant in FL is Tampa.
